# Approche Deep Learning

Nous proposons une approche élégante en utilisant les réseaux de neurones convolutionnels directements appliqués aux images issues des plot des landmarks.
Cette approche élégante sera conduite de manière la plus optimisée possible.

Il y a aura  deux types d'images :
- les plot avec des points noirs
- les plot avec les points colorés délimitant les différentes zones du visage (annotation manuelle des images)

Ce notebook comprendra une approche directe (pas d'augmentation, pas/peu d'optimisation d'hyperparametres) et un approche avancée (data augmentation, architecture optimisée) et si le temps le permet du fine tuning ou transfert learning vers d'autres backbones plus avancés.

## Librairies

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

# DATA IS IN BLACK AND WHITE
### Data generation and preprocessing
## NO DATA AUGMENTATION

In [ ]:
#### SUPER IMPORTANT TO EXECUTE OTHERWISE FACES MIGHT BE RGB
import save_face_img

save_face_img.create_faces(color_mode="bw")

In [ ]:
# Data Preparation NO AUGMENTATION
train_datagen =         ImageDataGenerator(rescale=1./255)
test_datagen =          ImageDataGenerator(rescale=1./255)

train_generator =       train_datagen.flow_from_directory(
                        r'../CK+_lands/images/train/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

validation_generator =  test_datagen.flow_from_directory(
                        r'../CK+_lands/images/val/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

### Building / Compiling / Fitting model

In [ ]:
# Model Building
model =     Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

# Compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

pat = 5 #this is the number of epochs with no improvment after which the training will stop
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

In [ ]:
import visualkeras
visualkeras.layered_view(model, legend=True, scale_xy=1)

### Fitting model

In [ ]:
history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=early_stopping) 

In [ ]:
# model.save('emotion_classifier_model.h5')  

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('epochs')
plt.ylabel('unit')
plt.legend()
plt.plot(history.history['loss'], label='loss')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('epochs')
plt.ylabel('unit')
plt.legend()
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()



In [ ]:
test_datagen =          ImageDataGenerator(rescale=1./255)

test_generator =        test_datagen.flow_from_directory(
                        r'../CK+_lands/images/test/',
                        target_size=(150, 150),
                        batch_size=1,
                        class_mode='categorical')

evaluation =            model.evaluate(test_generator)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

## WITH DATA AUGMENTATION

In [ ]:
# Data Preparation and augmentation
train_datagen_augmented =         ImageDataGenerator(rescale=1./255,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        rotation_range=15,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1)

test_datagen =          ImageDataGenerator(rescale=1./255)

train_generator_augmented =       train_datagen_augmented.flow_from_directory(
                        r'../CK+_lands/images/train/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

validation_generator =  test_datagen.flow_from_directory(
                        r'../CK+_lands/images/val/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

In [ ]:
# Model Building
model2 =     Sequential()
model2.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model2.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model2.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model2.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model2.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model2.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model2.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model2.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(6, activation='softmax'))

# Compilation
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()

In [ ]:
history = model2.fit(train_generator_augmented, epochs=100, validation_data=validation_generator) 

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('epochs')
plt.ylabel('unit')
plt.legend()
plt.plot(history.history['loss'], label='loss')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('epochs')
plt.ylabel('unit')
plt.legend()
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.tight_layout()



In [ ]:
test_datagen =          ImageDataGenerator(rescale=1./255)

test_generator =        test_datagen.flow_from_directory(
                        r'../CK+_lands/images/test/',
                        target_size=(150, 150),
                        batch_size=1,
                        class_mode='categorical')

evaluation =            model2.evaluate(test_generator)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

# DATA HAS COLORED ZONES (cf report figs)

In [ ]:
#### SUPER IMPORTANT TO EXECUTE OTHERWISE FACES MIGHT BE B&W
import save_face_img

save_face_img.create_faces(color_mode="rgb")

In [ ]:
# Data Preparation NO AUGMENTATION
train_datagen =         ImageDataGenerator(rescale=1./255)
test_datagen =          ImageDataGenerator(rescale=1./255)

train_generator =       train_datagen.flow_from_directory(
                        r'../CK+_lands/images/train/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

validation_generator =  test_datagen.flow_from_directory(
                        r'../CK+_lands/images/val/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

In [ ]:
# Model Building
model3 =     Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model3.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model3.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model3.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model3.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model3.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model3.add(Flatten())
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(6, activation='softmax'))

# Compilation
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model3.summary()

In [ ]:
from keras.callbacks import EarlyStopping

pat = 5 #this is the number of epochs with no improvment after which the training will stop
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

In [ ]:
history = model3.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=early_stopping) 

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('epochs')
plt.ylabel('unit')
plt.legend()
plt.plot(history.history['loss'], label='loss')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('epochs')
plt.ylabel('unit')
plt.legend()
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.tight_layout()



In [ ]:
test_datagen =          ImageDataGenerator(rescale=1./255)

test_generator =        test_datagen.flow_from_directory(
                        r'../CK+_lands/images/test/',
                        target_size=(150, 150),
                        batch_size=1,
                        class_mode='categorical')

evaluation =            model3.evaluate(test_generator)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

## Model tuning via GridSearch

In [ ]:
#libs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [ ]:
#create model
def create_model(activation='relu'):
    # for computational purposes we will use a very simple model
    model3 =     Sequential()
    model3.add(Conv2D(32, kernel_size=(3, 3), activation=activation,vinput_shape=(150, 150, 3)))
    model3.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
    model3.add(Flatten())
    model3.add(Dense(256, activation=activation))
    model3.add(Dropout(0.5))
    model3.add(Dense(6, activation='softmax'))

    # Compilation
    model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model3

In [ ]:
import tqdm
batch_size = 1
X_train, y_train = next(train_generator)
X_val, y_val = next(validation_generator)
for i in tqdm.tqdm(range(int(train_generator.n/batch_size)-1)): 
  img, label = next(train_generator)
  X_train = np.append(X_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
for i in tqdm.tqdm(range(int(validation_generator.n/1)-1)): 
  img, label = next(validation_generator)
  X_val = np.append(X_val, img, axis=0 )
  y_val = np.append(y_val, label, axis=0)


In [ ]:
## TOO LONG AND VERSION ISSUES WITH SCIKERAS AND PYTHON 8.
# model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0, callbacks=early_stopping)
# # define the grid search parameters
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# param_grid = dict(model__activation=activation)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
# grid_result = grid.fit(X_train, y_train)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))